[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/00-download-cord-19.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/00-download-cord-19.ipynb)

We start by downloading the CORD-19 dataset from Allen Institute for AI, either manually from [here](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html) or by running the code block below:

In [1]:
import requests

url = "https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2022-02-07.tar.gz"

with requests.get(url, stream=True) as r:
    with open('cord-19.tar.gz', 'wb') as fp:
        for chunk in r.iter_content(chunk_size=1024):
            fp.write(chunk)

Both methods create a *tar.gz* file, we need to extract the files stored within here.

In [2]:
import tarfile

with tarfile.open('cord-19.tar.gz', 'r:gz') as tar:
    tar.extractall()

This extracts a new directory named as the date of the dataset, in our case that is *2022-02-07*. In this directory we have a another *tar.gz* file called *document_parses.tar.gz*, which contains the data we will be using. Again, we extract this with `tarfile` again.

In [3]:
with tarfile.open('2022-02-07/document_parses.tar.gz') as tar:
    tar.extractall()

This does require **~59GB** of disk space. If you cannot extract the full amount due to space restrictions, it's completely fine to use a significantly smaller set of documents.

After extraction we should find a new directory *document_parses* that contains two file directories:

* *pdf_json* (34.3GB)
* *pmc_json* (24.6GB)

Both are large enough for our use-case, so we will stick with just *pdf_json*. Let's load a file from here to understand the format being used.

In [4]:
from pathlib import Path

paths = [
    str(path) for path in Path('document_parses/pdf_json').glob('*.json')
]
print(str(len(paths))+"\n"+paths[0])

338451
document_parses\pdf_json\0000028b5cc154f68b8a269f6578f21e31f62977.json


We have 338,450 PDF files in JSON format. We will take the first file to understand their structure.

In [5]:
import json

with open(paths[0], 'r') as fp:
    data = json.load(fp)

In [6]:
data

{'paper_id': '0000028b5cc154f68b8a269f6578f21e31f62977',
 'metadata': {'title': '"Multi-faceted" COVID-19: Russian experience',
  'authors': []},
 'abstract': [],
 'body_text': [{'text': 'According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remains in a position of advantage due to the later onset of the viral spread within the country since the worldwide disease outbreak.',
   'cite_spans': [],
   'ref_spans': [],
   'section': 'Editor'},
  {'text': 'The first step in "fighting" the epidemic was nationwide lock down on March 30 th , 2020.',
   'cite_spans': [],
   'ref_spans': [],
   'section': 'Editor'},
  {'text': 'Most of the multidisciplinary hospitals have been repurposed as dedicated COVID-19 centres, so the surgeons started working as infectious disease specialists. Such a reallocation of health care capacity results in the effective manag

In [7]:
data.keys()

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])

We can read the `'body_text'`...

In [8]:
data['body_text']

[{'text': 'According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remains in a position of advantage due to the later onset of the viral spread within the country since the worldwide disease outbreak.',
  'cite_spans': [],
  'ref_spans': [],
  'section': 'Editor'},
 {'text': 'The first step in "fighting" the epidemic was nationwide lock down on March 30 th , 2020.',
  'cite_spans': [],
  'ref_spans': [],
  'section': 'Editor'},
 {'text': 'Most of the multidisciplinary hospitals have been repurposed as dedicated COVID-19 centres, so the surgeons started working as infectious disease specialists. Such a reallocation of health care capacity results in the effective management of this epidemiological problem 1 . The staff has undergone on-line 36-hour training course to become qualified in coronavirus infection treatment.',
  'cite_spans': [],
  'ref_sp

In [9]:
len(data['body_text'])

13

We have a lot of text data here, *60* of these larger passages, let's try extracting them.

In [10]:
body_text = [
    line['text'] for line in data['body_text']
]
body_text[:3]

['According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remains in a position of advantage due to the later onset of the viral spread within the country since the worldwide disease outbreak.',
 'The first step in "fighting" the epidemic was nationwide lock down on March 30 th , 2020.',
 'Most of the multidisciplinary hospitals have been repurposed as dedicated COVID-19 centres, so the surgeons started working as infectious disease specialists. Such a reallocation of health care capacity results in the effective management of this epidemiological problem 1 . The staff has undergone on-line 36-hour training course to become qualified in coronavirus infection treatment.']

That's perfect, we now have a very large repository of COVID related passages of text. We will use this logic when loading the dataset in our use-cases. All directories other than *document_parses* can be removed.